In [ ]:
pip install openrouteservice

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point, mapping
from shapely.ops import unary_union
import openrouteservice as ors

# Exemplo de pontos de alagamento (viriam da API real depois)
flood_points = [
    (-23.5505, -46.6333),  # Centro de SP
    (-23.5375, -46.6250),  # Bela Vista
    (-23.5480, -46.6200)   # Sé
]

# Mostrando os dados como DataFrame
df_floods = pd.DataFrame(flood_points, columns=["Latitude", "Longitude"])
df_floods


,Latitude,Longitude
0,-23.5505,-46.6333
1,-23.5375,-46.6250
2,-23.5480,-46.6200


In [ ]:
# Gerar polígonos de até 1 km² (cerca de 600m de raio)
RAIO_KM = 0.3
raio_graus = RAIO_KM / 111.0  # Aproximado

poligonos = [Point(lon, lat).buffer(raio_graus) for lat, lon in flood_points]
poligono_unico = unary_union(poligonos)

avoid_geojson = {
    "type": "MultiPolygon",
    "coordinates": mapping(poligono_unico)["coordinates"]
}


In [ ]:
# Origem e destino ajustados (dentro de São Paulo)
# Exemplo: da Vila Mariana até a Penha
dest = [-46.6331, -23.5543]  # Vila Mariana
orig = [-46.6191, -23.5491]  # Penha

In [ ]:
# Conectar com sua chave do ORS
client = ors.Client(key="-")  # Substitua pela sua chave!

# Solicitar rota evitando os alagamentos
try:
    rota_geojson = client.directions(
        coordinates=[orig, dest],
        profile='driving-car',
        format='geojson',
        options={'avoid_polygons': avoid_geojson}
    )
    print("✅ Rota obtida com sucesso!")
except Exception as e:
    rota_geojson = None
    print("❌ Erro ao solicitar rota:", e)


❌ Erro ao solicitar rota: 404 ({'error': {'code': 2009, 'message': 'Route could not be found - Unable to find a route between points 1 (-46.6191000 -23.5491000) and 2 (-46.6331000 -23.5543000).'}, 'info': {'engine': {'build_date': '2025-05-06T08:31:01Z', 'graph_version': '1', 'graph_date': '2025-06-01T13:27:25Z', 'version': '9.2.0'}, 'timestamp': 1749242459669}})


In [ ]:
mapa = folium.Map(location=[-23.55, -46.62], zoom_start=12)

# Adicionar alagamentos como círculos de calor
heat_data = [[lat, lon, 1] for lat, lon in flood_points]
HeatMap(heat_data, radius=30, blur=20, gradient={0.4: "orange", 0.6: "red"}).add_to(mapa)

# Adicionar os polígonos de exclusão
folium.GeoJson(
    avoid_geojson,
    name="Zonas Alagadas",
    style_function=lambda *_: {
        "fillColor": "red",
        "color": "red",
        "weight": 1,
        "fillOpacity": 0.3
    },
).add_to(mapa)

# Adicionar origem e destino
folium.Marker(location=orig[::-1], popup="Origem", icon=folium.Icon(color="green")).add_to(mapa)
folium.Marker(location=dest[::-1], popup="Destino", icon=folium.Icon(color="orange")).add_to(mapa)

# Adicionar a rota, se obtida
if rota_geojson:
    folium.GeoJson(
        rota_geojson,
        name="Rota Segura",
        style_function=lambda *_: {"color": "blue", "weight": 4}
    ).add_to(mapa)

folium.LayerControl(collapsed=False).add_to(mapa)
mapa
